In [1]:
import numpy as np
import pandas as pd
import seaborn as sns

from tennis_model_predict import get_model_win_proba
from tennis_model_predict import general_elo_dict, hard_dict, ts_dict, hand_dict

from trueskillthroughtime import Game, Player

from elo_funcs import elo_predict

from sklearn.metrics import accuracy_score, roc_auc_score, brier_score_loss
from sklearn.calibration import calibration_curve

C:\Users\jyoung\Projects\tennis_project\tennis_model_predict.py:54: DtypeWarning: Columns (8,16) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(f'{data_folder}/processed_apt_with_ts.csv', parse_dates=['tourney_date'])


All betting market data sourced from: http://www.tennis-data.co.uk/alldata.php

In [2]:
ts_file = r'C:\Users\jyoung\Projects\tennis_project\tennis_data\tennis_atp-master\processed_apt_with_ts.csv'

ts_df = pd.read_csv(ts_file, parse_dates=['tourney_date'])
# ts_df['winner_last'] = [x.split(' ')[-1] for x in ts_df['winner_name']]
# ts_df['loser_name'] = [x.split(' ')[-1] for x in ts_df['loser_name']]
ts_df['month'] = ts_df['tourney_date'].dt.month
ts_df['year'] = ts_df['tourney_date'].dt.year

C:\Users\jyoung\AppData\Local\Temp\ipykernel_19176\1959857988.py:3: DtypeWarning: Columns (8,16) have mixed types. Specify dtype option on import or set low_memory=False.
  ts_df = pd.read_csv(ts_file, parse_dates=['tourney_date'])


In [3]:
xls_file = r'C:\Users\jyoung\Projects\tennis_project\tennis_data\tennis_atp_betting_results\2022.xlsx'

df = pd.read_excel(xls_file, parse_dates=['Date'])
df.columns = df.columns.str.lower()
df['month'] = df['date'].dt.month
df['year'] = df['date'].dt.year

In [4]:
df['winner_last'] = [x.split(' ')[0] for x in df['winner']]
df['loser_last'] = [x.split(' ')[0] for x in df['loser']]

In [23]:
df

,atp,location,tournament,date,series,court,surface,round,best of,winner,...,psw,psl,maxw,maxl,avgw,avgl,month,year,winner_last,loser_last
0,1,Adelaide,Adelaide International 1,2022-01-03,ATP250,Outdoor,Hard,1st Round,3.0,Kwon S.W.,...,1.70,2.26,1.76,2.50,1.64,2.22,1,2022,Kwon,Nishioka
1,1,Adelaide,Adelaide International 1,2022-01-03,ATP250,Outdoor,Hard,1st Round,3.0,Monteiro T.,...,2.49,1.60,2.60,1.62,2.38,1.57,1,2022,Monteiro,Altmaier
2,1,Adelaide,Adelaide International 1,2022-01-03,ATP250,Outdoor,Hard,1st Round,3.0,Djere L.,...,1.81,2.10,1.83,2.26,1.70,2.12,1,2022,Djere,Carballes
3,1,Adelaide,Adelaide International 1,2022-01-03,ATP250,Outdoor,Hard,1st Round,3.0,Johnson S.,...,2.21,1.74,2.30,1.80,2.10,1.72,1,2022,Johnson,Vukic
4,1,Adelaide,Adelaide International 1,2022-01-04,ATP250,Outdoor,Hard,1st Round,3.0,Moutet C.,...,2.66,1.53,2.81,1.54,2.64,1.47,1,2022,Moutet,Rune
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2627,68,Turin,Masters Cup,2022-11-18,Masters Cup,Indoor,Hard,Round Robin,NaN,Djokovic N.,...,1.37,3.36,1.39,3.55,1.34,3.25,11,2022,Djokovic,Medvedev
2628,68,Turin,Masters Cup,2022-11-18,Masters Cup,Indoor,Hard,Round Robin,NaN,Rublev A.,...,2.63,1.55,2.63,1.63,2.39,1.57,11,2022,Rublev,Tsitsipas
2629,68,Turin,Masters Cup,2022-11-19,Masters Cup,Indoor,Hard,Semifinals,NaN,Djokovic N.,...,1.24,4.53,1.28,4.75,1.23,4.14,11,2022,Djokovic,Fritz
2630,68,Turin,Masters Cup,2022-11-19,Masters Cup,Indoor,Hard,Semifinals,NaN,Ruud C.,...,2.35,1.67,2.35,1.75,2.18,1.69,11,2022,Ruud,Rublev


In [5]:
def true_skill_win_proba(player_1=None, player_2=None):

  player_1 = Player(ts_dict[player_1][-1][1])
  player_2 = Player(ts_dict[player_2][-1][1])

  return Game([[player_1], [player_2]]).evidence

In [87]:
test_df = (
    df
    .merge(ts_df,
           how='left',
           left_on=['year', 'month', 'wrank', 'lrank'],
           right_on=['year', 'month', 'winner_rank', 'loser_rank'])
)

test_df.dropna(subset=['winner_ts_win_prob', 'avgw', 'b365w'], inplace=True)

test_df['avgw_prob'] = 1 / (test_df['avgw'])
test_df['avgl_prob'] = 1 / (test_df['avgl'])
test_df['b365w_prob'] = 1 / (test_df['b365w'])

test_df['win_prob_diff'] = test_df['avgw_prob'].sub(test_df['winner_ts_win_prob'])
test_df['win_prob_diff_bin'] = (
    pd.cut(
        test_df['win_prob_diff'],
        bins=[-0.3, -0.2, -0.1, 0, 0.1, 0.2, 0.3])
)

win_condition = (
    ((test_df['avgw_prob'] >= 0.5) & (test_df['winner_ts_win_prob'] >= 0.5)) |
    ((test_df['avgw_prob'] < 0.5) & (test_df['winner_ts_win_prob'] < 0.5))
)

test_df['win_agreement'] = np.where(win_condition, 1, 0)

In [165]:
def return_losses(x):
    
    a = brier_score_loss([1] * len(x), x['winner_ts_win_prob'])
    b = brier_score_loss([1] * len(x), x['avgw_prob'])
    
    df_ = pd.DataFrame({'ts_loss' : a, 'market_loss' : b}, index=[0])
    
    return df_

In [166]:
def return_accuracies(x):
    
    "can return puzzling inconsistencies because 0.5 is rounded to 0"
    
    a = accuracy_score([1] * len(x), (x['winner_ts_win_prob']).round(0))
    b = accuracy_score([1] * len(x), (x['avgw_prob']).round(0))
    
    x['ts_accuracy'] = a
    x['market_accuracy'] = b
    
    return x[['ts_accuracy', 'market_accuracy']].reset_index(drop=True).drop_duplicates(subset=['ts_accuracy'])

In [167]:
test_df.groupby('surface_y').apply(return_losses)

,,ts_loss,market_loss
surface_y,,,
Clay,0,0.185217,0.176232
Grass,0,0.197555,0.185652
Hard,0,0.174278,0.177077


In [143]:
test_df.groupby(['surface_y', 'round_y']).apply(return_losses).reset_index()

,surface_y,round_y,level_2,ts_loss,market_loss
0,Clay,1.0,0,0.165530,0.159116
1,Clay,2.0,0,0.166919,0.146755
2,Clay,3.0,0,0.196127,0.187112
3,Clay,4.0,0,0.184901,0.178616
4,Clay,5.0,0,0.190539,0.172318
5,Clay,6.0,0,0.182222,0.182899
6,Clay,7.0,0,0.196722,0.246965
7,Grass,1.0,0,0.149080,0.139426
8,Grass,2.0,0,0.181694,0.181314
9,Grass,3.0,0,0.214053,0.199222


In [134]:
bet_set = test_df[test_df['win_agreement'] == 0].copy()

bankroll = 1000
bankroll_steps = []

for a, b, c in zip(bet_set['avgw'], bet_set['avgl'], bet_set['winner_ts_win_prob']):
    
    if c >= 0.5:
        
        bankroll += (100 * a) - 100
        bankroll_steps.append(['w', bankroll])
        print(a, b, c, "W")
        
    else:
        bankroll -= 100
        bankroll_steps.append(['l', bankroll])
        print(a, b, c, bankroll)

print(bankroll)

2.38 1.57 0.5601974909396072 1138.0
2.1 1.72 0.5345326614413717 1248.0
2.64 1.47 0.5354893988287746 1412.0
2.04 1.77 0.5037584624879813 1516.0
2.89 1.41 0.6800806118185162 1705.0
1.9 1.88 0.3849346662408056 1605.0
1.56 2.42 0.4698490201056837 1505.0
1.92 1.87 0.4385225810812262 1405.0
2.77 1.43 0.5773626368859105 1582.0
2.11 1.71 0.6445557324590618 1693.0
2.07 1.75 0.5535243416040334 1800.0
2.37 1.57 0.5420618839768889 1937.0
2.14 1.71 0.5636962927893067 2051.0
2.03 1.79 0.6223824189379066 2154.0
1.88 1.92 0.4723339747740511 2054.0
1.87 1.94 0.4560048193062647 1954.0
1.74 2.09 0.4330439433631365 1854.0
2.18 1.68 0.5246164901188031 1972.0
1.65 2.23 0.4804824506743206 1872.0
2.11 1.73 0.5179894406612124 1983.0
1.85 1.96 0.4914663672054566 1883.0
2.51 1.52 0.6478949788443544 2034.0
2.09 1.73 0.5653276914411798 2143.0
1.77 2.07 0.3910062763568851 2043.0
1.68 2.18 0.4916633429865009 1943.0
2.0 1.81 0.345695881517197 1843.0
2.53 1.52 0.625025584346562 1996.0
2.16 1.69 0.6039023541452022 2112

In [135]:
len(test_df)

2427

In [137]:
bankroll_steps

[['w', 1138.0],
 ['w', 1248.0],
 ['w', 1412.0],
 ['w', 1516.0],
 ['w', 1705.0],
 ['l', 1605.0],
 ['l', 1505.0],
 ['l', 1405.0],
 ['w', 1582.0],
 ['w', 1693.0],
 ['w', 1800.0],
 ['w', 1937.0],
 ['w', 2051.0],
 ['w', 2154.0],
 ['l', 2054.0],
 ['l', 1954.0],
 ['l', 1854.0],
 ['w', 1972.0],
 ['l', 1872.0],
 ['w', 1983.0],
 ['l', 1883.0],
 ['w', 2034.0],
 ['w', 2143.0],
 ['l', 2043.0],
 ['l', 1943.0],
 ['l', 1843.0],
 ['w', 1996.0],
 ['w', 2112.0],
 ['w', 2490.0],
 ['l', 2390.0],
 ['w', 2511.0],
 ['w', 2664.0],
 ['l', 2564.0],
 ['w', 2676.0],
 ['w', 2832.0],
 ['l', 2732.0],
 ['w', 2856.0],
 ['l', 2756.0],
 ['l', 2656.0],
 ['l', 2556.0],
 ['l', 2456.0],
 ['w', 2558.0],
 ['l', 2458.0],
 ['l', 2358.0],
 ['l', 2258.0],
 ['l', 2158.0],
 ['l', 2058.0],
 ['w', 2178.0],
 ['w', 2343.0],
 ['w', 2453.0],
 ['w', 2573.0],
 ['w', 2688.0],
 ['l', 2588.0],
 ['l', 2488.0],
 ['w', 2608.0],
 ['w', 2726.0],
 ['l', 2626.0],
 ['l', 2526.0],
 ['w', 2662.0],
 ['w', 2804.0],
 ['l', 2704.0],
 ['w', 2867.0],
 ['w', 2

In [110]:
(bet_set['winner_ts_win_prob'] >= 0.5).mean()

0.5525606469002695